In [ ]:

# Part 1: 기본 설정 및 라이브러리 임포트
import os
import glob
import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("TensorFlow Version:", tf.__version__)

In [ ]:
# GPU가 사용 가능한지 확인
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import numpy as np
from PIL import Image
from tensorflow.keras import layers


# 데이터 전처리 함수 정의

# --- 2-1. 이미지 로드 및 기본 전처리 ---
def load_and_preprocess_image(path, size=(256, 256)):
    img = Image.open(path).convert('L').resize(size, Image.LANCZOS)
    return np.array(img, dtype=np.float32) / 255.0

# --- 2-2. 데이터 증강 ---
data_augmentation_pipeline = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
], name="geometric_augmentation")

def augment_brightness_contrast(image):
    image_with_channel = image[..., tf.newaxis]
    image_aug = tf.image.random_brightness(image_with_channel, max_delta=0.15)
    image_aug = tf.image.random_contrast(image_aug, lower=0.8, upper=1.2)
    return tf.squeeze(image_aug)

# --- 2-3. 노이즈 추가 함수 ---
def add_noise_snr_np(image, snr_db):
    signal_power = np.mean(np.square(image))
    snr_linear = 10 ** (snr_db / 10.0)
    noise_power = signal_power / snr_linear
    noise_sigma = np.sqrt(noise_power)
    noise = np.random.normal(0, noise_sigma, image.shape).astype(np.float32)
    return np.clip(image + noise, 0.0, 1.0).astype(np.float32)

def add_salt_and_pepper_noise(image, amount=0.05):
    noisy_image = np.copy(image)
    # Salt
    num_salt = np.ceil(amount * image.size * 0.5).astype(int)
    coords = tuple(np.random.randint(0, i-1, num_salt) for i in image.shape if i > 1)
    if coords and coords[0].size > 0: noisy_image[coords] = 1.0
    # Pepper
    num_pepper = np.ceil(amount * image.size * 0.5).astype(int)
    coords = tuple(np.random.randint(0, i-1, num_pepper) for i in image.shape if i > 1)
    if coords and coords[0].size > 0: noisy_image[coords] = 0.0
    return noisy_image

def add_burst_noise(image, burst_size_factor=0.2, intensity=0.8):
    noisy_image = np.copy(image)
    h, w = image.shape
    burst_h, burst_w = int(h * burst_size_factor), int(w * burst_size_factor)
    if h <= burst_h or w <= burst_w: return noisy_image
    start_y = np.random.randint(0, h - burst_h)
    start_x = np.random.randint(0, w - burst_w)
    burst_noise = np.random.normal(0, intensity, (burst_h, burst_w)).astype(np.float32)
    burst_area = noisy_image[start_y:start_y+burst_h, start_x:start_x+burst_w]
    noisy_area = np.clip(burst_area + burst_noise, 0.0, 1.0)
    noisy_image[start_y:start_y+burst_h, start_x:start_x+burst_w] = noisy_area
    return noisy_image

print("All preprocessing functions are defined.")

In [ ]:
# Part 3: 파일 데이터 전처리 및 로드
import glob
from tqdm import tqdm

# --- 3-1. 원본 이미지 경로 정의 및 로드 ---
all_image_paths = ['data/mona_lisa.jpg'] + sorted(glob.glob('data/sample*.jpg'))
class_names = [f"Mona Lisa {i} ({'Original' if i==0 else f'Parody_{i}'})" for i in range(len(all_image_paths))]
try:
    original_images = [load_and_preprocess_image(p) for p in all_image_paths]
    print(f"Successfully loaded {len(original_images)} images for {len(class_names)} classes.")
except Exception as e:
    print(f"Error during image loading: {e}")

# --- 3-2. 최종 데이터셋 생성 함수 (조건부 입력 포함) ---
def create_conditional_dataset(images, samples_per_class=100, max_sigma=50.0):
    X_noisy, X_noise_map, y_clean, y_class = [], [], [], []
    noise_types = ['gaussian', 'salt_pepper', 'burst']
    
    with tqdm(total=len(images) * samples_per_class, desc="Generating Conditional Dataset") as pbar:
        for class_idx, original_image in enumerate(images):
            for _ in range(samples_per_class):
                # Augmentation
                augmented_image = data_augmentation_pipeline(original_image[np.newaxis, ..., np.newaxis], training=True)
                augmented_image = np.squeeze(augmented_image)
                augmented_image = augment_brightness_contrast(augmented_image).numpy()
                clean_augmented_image = np.clip(augmented_image, 0.0, 1.0)

                # Noise Addition
                noise_type = np.random.choice(noise_types)
                sigma_equivalent = 0.0 # 노이즈 맵에 기록할 값
                
                if noise_type == 'gaussian':
                    snr = np.random.choice([-10, -20, -30])
                    # SNR을 sigma로 근사 변환 (단순화된 방식)
                    signal_power = np.mean(np.square(clean_augmented_image))
                    noise_power = signal_power / (10 ** (snr / 10.0))
                    sigma_equivalent = np.sqrt(noise_power) * 255 # 0~255 스케일의 sigma
                    noisy_img = add_noise_snr_np(clean_augmented_image, snr)
                elif noise_type == 'salt_pepper':
                    amount = np.random.uniform(0.05, 0.2)
                    noisy_img = add_salt_and_pepper_noise(clean_augmented_image, amount=amount)
                    sigma_equivalent = amount * 50 # S&P 강도를 sigma처럼 맵핑 (경험적)
                else: # burst
                    size = np.random.uniform(0.2, 0.5)
                    intensity = np.random.uniform(0.7, 1.0)
                    noisy_img = add_burst_noise(clean_augmented_image, burst_size_factor=size, intensity=intensity)
                    sigma_equivalent = size * 50 # Burst 강도를 sigma처럼 맵핑 (경험적)

                # Noise Map 생성
                noise_map = np.full(clean_augmented_image.shape, sigma_equivalent / max_sigma, dtype=np.float32)
                
                X_noisy.append(noisy_img)
                X_noise_map.append(noise_map)
                y_clean.append(clean_augmented_image)
                y_class.append(class_idx)
                pbar.update(1)

    indices = np.arange(len(X_noisy))
    np.random.shuffle(indices)

    X_noisy = np.array(X_noisy)[indices][..., np.newaxis]
    X_noise_map = np.array(X_noise_map)[indices][..., np.newaxis]
    y_clean = np.array(y_clean)[indices][..., np.newaxis]
    y_class = np.array(y_class)[indices]
    
    return [X_noisy, X_noise_map], [y_clean, y_class]

# --- 3-3. 데이터셋 생성 실행 ---
[X_noisy_data, X_noise_map_data], [y_restore_data, y_classify_data] = create_conditional_dataset(
    original_images, samples_per_class=100
)

print("\n--- Dataset Creation Complete ---")
print(f"Input Noisy Image Shape: {X_noisy_data.shape}")
print(f"Input Noise Map Shape: {X_noise_map_data.shape}")
print(f"Target Clean Image Shape: {y_restore_data.shape}")
print(f"Target Class Label Shape: {y_classify_data.shape}")

In [ ]:


def build_conditional_multitask_unet(input_shape, num_classes):
    
    def conv_block(input_tensor, num_filters):
        x = layers.Conv2D(num_filters, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-5))(input_tensor)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(num_filters, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-5))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        return x

    def decoder_block(input_tensor, skip_tensor, num_filters):
        x = layers.Conv2DTranspose(num_filters, 2, strides=2, padding='same')(input_tensor)
        x = layers.concatenate([x, skip_tensor])
        x = conv_block(x, num_filters)
        return x

    # --- 두 개의 입력 정의 ---
    image_input = layers.Input(shape=input_shape, name="image_input")
    noise_map_input = layers.Input(shape=input_shape, name="noise_map_input")
    
    # --- 입력 결합 ---
    concatenated_input = layers.concatenate([image_input, noise_map_input]) # Shape: (H, W, 2)
    
    # Encoder
    c1 = conv_block(concatenated_input, 16)
    p1 = layers.MaxPooling2D(2)(c1)
    c2 = conv_block(p1, 32)
    p2 = layers.MaxPooling2D(2)(c2)
    c3 = conv_block(p2, 64)
    p3 = layers.MaxPooling2D(2)(c3)
    
    # Bottleneck
    b = conv_block(p3, 128)
    
    # Decoder
    d3 = decoder_block(b, c3, 64)   
    d2 = decoder_block(d3, c2, 32)
    d1 = decoder_block(d2, c1, 16)
    
    # --- 두 개의 헤드 정의 ---
    restoration_head = layers.Conv2D(1, 1, activation='sigmoid', name="restoration_output")(d1)
    
    flat = layers.GlobalAveragePooling2D()(b)
    dense1 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(flat)
    dropout = layers.Dropout(0.5)(dense1)
    classification_head = layers.Dense(num_classes, activation='softmax', name="classification_output")(dropout)
    
    model = Model(inputs=[image_input, noise_map_input], outputs=[restoration_head, classification_head])
    return model

# 모델 생성
num_total_classes = len(class_names)
model = build_conditional_multitask_unet(input_shape=(256, 256, 1), num_classes=num_total_classes)
model.summary()